#### ResNet

##### 第一部分 python库调用

In [36]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from tqdm import tqdm

##### 第二部分 数据预处理
* RandomHorizontalFlip()：以50%概率水平翻转图像，适合处理对称性图像和数据。对于非对称数据，例如鞋子，可能会产生不真实样本，但是可以提升模型的泛化能力。
* RandomCrop(28, padding=4)：在28x28的图像周围填充4个像素，变成36x36，再随机裁剪出28x28的图像。模拟物体位置变化的情况，增强模型对物体位置的鲁棒性。
* ToTensor()：将pil图像或者numpy数组转为torch张量，并且自动将像素值从255缩放到1，调整维度为1x28x28.
* Normalize()：将[0,1]映射到[-1,1]。标准化数据分布，加速模型收敛。

In [37]:
# 数据预处理
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(28,padding=4),
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,),(0.5,))
])

##### 第三部分 ResNet基本模块
关键点：
* 残差结构：通过'out += self.shortcut(x)'实现跳跃连接，主路径学习残差f(x)=h(x)-x。
* 维度匹配：当输入输出通道数不同或需要下采样时，使用1x1卷积调整路径。

In [38]:
# 定义ResNet基本模块（残差块）
class BasicBlock(nn.Module):
    expansion = 1 # 通道扩展系数，用于控制输出通道数

    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__() # 调用父类nn.Module的初始化方法

        # 第一个卷积层（3x3）
        self.conv1 = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size = 3,
            stride = stride, # 步长为1，保持原始尺寸
            padding = 1, # 填充为1，在3x3卷积下，保持尺寸不变
            bias = False
        )

        # 第一个批量归一化层
        self.bn1 = nn.BatchNorm2d(out_channels)

        # 第二个卷积层（3x3）
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels,
            kernel_size = 3,
            stride = 1,
            padding = 1,
            bias = False
        )

        # 第二个批量归一化层
        self.bn2 = nn.BatchNorm2d(out_channels)

        # 捷径连接
        self.shortcut = nn.Sequential()

        # 如果输入通道数与输出通道数不一致，需要调整捷径连接
        if stride != 1 or in_channels != self.expansion * out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(
                    in_channels,
                    self.expansion * out_channels,
                    kernel_size = 1,
                    stride = stride,
                    bias = False
                ),
                nn.BatchNorm2d(self.expansion * out_channels)
            )

    def forward(self, x):
        # 主路径
        out = torch.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))

        # 残差连接：主路径+捷径路径
        out += self.shortcut(x)

        # 最终激活
        out = torch.relu(out)
        return out

##### 第四部分 定义完整的ResNet网络

In [39]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super().__init__()
        self.in_channels = 64 # 初始化输入通道数

        # 初始卷积层
        self.conv1 = nn.Conv2d(
            1, #输入通道为1，适应mnist
            64, #输出通道为64
            kernel_size = 3,
            stride = 1,
            padding = 1,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(64) # 64通道的批量归一化层

        # 四个层级，对应不同空间尺度
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1) #尺度维持
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2) #下采样
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2) #下采样
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2) #下采样

        # 全局平均赤化层，将特征图片压缩到1x1
        self.avgpool = nn.AdaptiveAvgPool2d((1,1))

        # 全连接层
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        """ 构建包含多个残差块的层级
        Args:
            block:     残差块类型（BasicBlock/Bottleneck）
            out_channels: 该层目标输出通道数
            num_blocks:   该层包含的残差块数量
            stride:       第一个残差块的步长（控制下采样）
        """
        # 生成步长列表，仅第一个块可能下采样
        strides = [stride] + [1] * (num_blocks-1)

        layers = [] # 保存该层所有残差块
        for stride in strides:
            # 创建残差块实例
            layers.append(block(
                self.in_channels,
                out_channels,
                stride
            ))
            # 更新下一个残差块的输入通道数
            self.in_channels = out_channels * block.expansion

        # 返回该层
        return nn.Sequential(*layers)
    
    # 前向传播
    def forward(self, x):
        # 初始卷积+bn+relu
        out = torch.relu(self.bn1(self.conv1(x)))

        # 通过4个layer
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)

        # 全局平均池化
        out = self.avgpool(out)

        # 展平层
        out = out.view(out.size(0), -1)

        # 全连接层
        out = self.fc(out)
        return out

In [40]:
# 输出测试
x = torch.randn(1, 1, 28, 28)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ResNet(BasicBlock, [2, 2, 2, 2]).to(device)
x = model(x.to(device))
x.shape

torch.Size([1, 10])

##### 第五部分 训练函数

In [41]:
# 训练函数
def train(model, device, trainloader, optimizer, criterion, epoch):
    # 设置模型为训练模式
    model.train()
    # 初始化累计损失和正确预测数
    total_loss = 0
    correct = 0
    # 设置进度条
    progress_bar = tqdm(trainloader, desc=f'Epoch {epoch}')

    # 遍历数据加载器，每次迭代获取一个batch
    for inputs, targets in progress_bar:
        # 将数据移动到指定设备
        inputs, targets = inputs.to(device), targets.to(device)
        # 梯度清零
        optimizer.zero_grad()
        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, targets)
        # 反向传播：计算梯度（链式法则）
        loss.backward()
        # 更新参数
        optimizer.step()

        # 累加损失
        total_loss += loss.item()
        # 计算预测值（dim=1取每行最大值索引）
        _, predicted = outputs.max(dim=1)
        # 累加正确预测数（eq比较预测与真实标签，sum求和）
        correct += predicted.eq(targets).sum().item()

        # 更新进度条显示信息
        progress_bar.set_postfix({
            'Loss': f"{total_loss/(progress_bar.n+1):.3f}",
            'Acc': f"{100.*correct/((progress_bar.n+1)*inputs.size(0)):.1f}%"
        })
    # 返回：平均损失（总损失/总batch数），总体准确率（正确数/总样本数）
    return total_loss / len(trainloader), correct / len(trainloader.dataset)

##### 第六部分 测试函数
值得注意，这里使用no_grad()，禁用梯度计算（节省内存，提升推理速度）。这在测试过程中是值得的。

In [42]:
def test(model, device, testloader, criterion):
    # 将模型设置为评估模式
    model.eval()
    # 初始化损失和正确预测数
    total_loss = 0
    correct = 0

    # 禁用梯度计算（节省内存，提升推理速度）
    with torch.no_grad():
        # 遍历测试数据
        for inputs, targets in testloader:
            # 将数据移动到指定设备
            inputs, targets = inputs.to(device), targets.to(device)
            # 前向传播
            outputs = model(inputs)
            # 计算损失
            loss = criterion(outputs, targets)
            # 累加损失
            total_loss += loss.item()
            _, predicted = outputs.max(dim=1)
            correct += predicted.eq(targets).sum().item()

    acc = 100. * correct / len(testloader.dataset)
    print(f'\nTest Loss: {total_loss/len(testloader):.3f}, Test Acc: {acc:.1f}%')
    return acc

##### 第七部分 主函数
主函数部分主要包含以下几个部分：
* 参数配置：配置一些全局公用的参数，例如batch_size等。
* 加载数据：进行数据集的加载。
* 模型初始化：设置初始化的模型，定义损失函数、优化器等。
* 训练循环

这里对scheduler动态调整学习率进行简单介绍。其核心参数如下：
* optimizer：绑定的优化器对象（如Adam/SGD）
* mode：监控指标的优化方向：'max'（越大越好，如准确率）或 'min'（越小越好，如损失）
* patience：触发学习率降低前允许指标停滞的epoch数
* factor (默认0.1)：学习率缩放因子（新LR = 旧LR × factor）
* threshold (默认1e-4)：指标变化的阈值（超过此值才视为"提升"）
* min_lr (默认0)：学习率下限（LR不会低于此值）

In [43]:
if __name__ == '__main__':
    # 配置参数
    batch_size = 256 # 每个批次包含256个样本
    num_workers = 0 # 加载数据的子进程数,Windows必须为0，其他系统可以设置为cpu核心数

    # 数据加载
    # 训练集配置
    trainset = torchvision.datasets.FashionMNIST(
        root = './data',
        train = True, # 加载训练集
        download = True, # 自动下载数据集
        transform = train_transform # 训练数据增强，需要提前定义，包含随机翻转、裁剪等
    )
    trainloader = DataLoader(
        trainset,
        batch_size = batch_size,
        shuffle = True, # 打乱数据顺序
        num_workers = num_workers
    )

    # 测试集配置
    testset = torchvision.datasets.FashionMNIST(
        root = './data',
        train = False,
        download = True,
        transform = test_transform
    )
    testloader = DataLoader(
        testset,
        batch_size = batch_size,
        shuffle = False, # 不需要打乱数据
        num_workers = num_workers
    )

    # 模型初始化和训练初始化
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # 优先使用GPU
    model = ResNet(BasicBlock, [2, 2, 2, 2]).to(device) # 初始化模型
    criterion = nn.CrossEntropyLoss() # 损失函数
    optimizer = optim.Adam(
        model.parameters(),
        lr=0.001, # 初始学习率
        weight_decay=1e-4 # L2正则化系数（防止过拟合）
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, 
        mode='max', # 根据准确率最大化调整学习率
        patience=3 # 连续3个epoch指标未提升则降低LR
    )

    # 训练循环
    best_acc = 0.0
    for epoch in range(10):
        train_loss, train_acc = train(model, device, trainloader, optimizer, criterion, epoch)
        test_acc = test(model, device, testloader, criterion)
        scheduler.step(test_acc)

        if test_acc > best_acc:
            best_acc = test_acc
            torch.save(model.state_dict(), 'best_model.pth')
            print(f"New best model saved! Accuracy: {best_acc:.2f}%")

    print(f"\nFinal Best Accuracy: {best_acc:.2f}%")

Epoch 0: 100%|██████████| 235/235 [00:12<00:00, 18.80it/s, Loss=0.592, Acc=209.4%]



Test Loss: 0.386, Test Acc: 86.3%
New best model saved! Accuracy: 86.33%


Epoch 1: 100%|██████████| 235/235 [00:12<00:00, 19.49it/s, Loss=0.361, Acc=231.6%]



Test Loss: 0.331, Test Acc: 88.1%
New best model saved! Accuracy: 88.06%


Epoch 2: 100%|██████████| 235/235 [00:12<00:00, 19.48it/s, Loss=0.308, Acc=235.9%]


...


Test Loss: 0.197, Test Acc: 93.0%
New best model saved! Accuracy: 92.96%


Epoch 8: 100%|██████████| 235/235 [00:11<00:00, 19.87it/s, Loss=0.175, Acc=250.4%]



Test Loss: 0.194, Test Acc: 93.4%
New best model saved! Accuracy: 93.36%


Epoch 9: 100%|██████████| 235/235 [00:11<00:00, 19.78it/s, Loss=0.171, Acc=251.7%]



Test Loss: 0.186, Test Acc: 93.5%
New best model saved! Accuracy: 93.52%

Final Best Accuracy: 93.52%
